In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QMessageBox
from PyQt5.QtWidgets import QFileDialog
import os, sys
import sqlite3
from tabulate import tabulate
import subprocess
import csv

In [2]:
class Content:
    def __init__(self,postno,content,commentno):
        self.pno = postno
        self.content = content
        self.cno = commentno

    def print_instance(self):
        print("Post No.: ",self.pno)
        print("Content: ",self.content)
        print("Comment No.:",self.cno)
        
def openfile(filename):
    fd = open(filename,"r+")
    output = list()
    if not fd:
        print ("Error opening file")
    else:
        while True:
            line = fd.readline()
            pno = ""
            content = ""
            cno = ""
            if line == "":
                break
            for i in range(len(line)):
                if line[i] == ";":
                    break
                pno += line[i]
            for j in range(i+1,len(line)):
                if line[j] == ";":
                    break
                content += line[j]
            for k in range(j+1,len(line)):
                cno += line[k]

            var = Content(int(pno), content, int(cno))
                
            output.append(var)        

    return output


def opendatabase(file):
    cwd=os.getcwd()
    path = cwd+"/"+file

    conn = sqlite3.connect(path, timeout = 10)
    #c = conn.cursor()

    return conn

def countoccurrence(conn, key, table):
    c = conn.cursor()
    res = c.execute("SELECT COUNT(content) FROM %s WHERE content='%s'"%(table, key.lower()))
    for row in res:
        return row[0]
    else:
        return 0

def findmax(conn,table):
    c = conn.cursor()
    res = c.execute("SELECT DISTINCT content, COUNT(content) as count FROM %s GROUP BY content ORDER BY count DESC"%table)
    count_list = list()
    flag = False
    for row in res:
        flag = True
        count_list.append(row)
    if flag == False:
        count_list.append(None)
        return -1

    table = tabulate(count_list, headers=["Title","Count"])
    print(table)
    return count_list[0], table

def updatedatabase(conn, List, table):
    c = conn.cursor()
    for i in range(len(List)):
        cursor = conn.execute("Select * from %s where content = '%s'"%(table, List[i].content))
        flag = False
        for row in cursor:
            if row[0].lower() == List[i].content.lower() and row[1] == List[i].cno and row[2] == List[i].pno:
                flag = True
                break
        else:
            flag = False

        if flag == True:
            pass
            #print("Duplicate comment: ",row)
        else:
            c.execute("INSERT INTO %s VALUES('%s',%d,%d)"%(table,List[i].content.lower(),int(List[i].cno),int(List[i].pno)))
            #print("Record inserted")

        conn.commit()


In [3]:
class Ui_Window1(object):
    def setupUi(self, Window1):
        Window1.setObjectName("Window1")
        Window1.resize(561, 481)
        self.centralwidget = QtWidgets.QWidget(Window1)
        self.centralwidget.setObjectName("centralwidget")
        self.title_label = QtWidgets.QLabel(self.centralwidget)
        self.title_label.setGeometry(QtCore.QRect(90, 20, 411, 91))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(20)
        font.setBold(True)
        font.setWeight(75)
        self.title_label.setFont(font)
        self.title_label.setObjectName("title_label")
        self.file_label = QtWidgets.QLabel(self.centralwidget)
        self.file_label.setGeometry(QtCore.QRect(150, 120, 111, 71))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(16)
        font.setBold(True)
        font.setWeight(75)
        self.file_label.setFont(font)
        self.file_label.setObjectName("file_label")
        self.file_button = QtWidgets.QPushButton(self.centralwidget)
        self.file_button.setGeometry(QtCore.QRect(270, 140, 131, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(12)
        self.file_button.setFont(font)
        self.file_button.setObjectName("file_button")
        self.submit_button = QtWidgets.QPushButton(self.centralwidget)
        self.submit_button.setGeometry(QtCore.QRect(230, 210, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.submit_button.setFont(font)
        self.submit_button.setObjectName("submit_button")
        self.exit_button = QtWidgets.QPushButton(self.centralwidget)
        self.exit_button.setGeometry(QtCore.QRect(450, 380, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.exit_button.setFont(font)
        self.exit_button.setObjectName("exit_button")
        Window1.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(Window1)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 561, 21))
        self.menubar.setObjectName("menubar")
        Window1.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(Window1)
        self.statusbar.setObjectName("statusbar")
        Window1.setStatusBar(self.statusbar)

        self.exit_button.clicked.connect(self.exit_button_pressed)
        self.submit_button.clicked.connect(lambda: self.submit_button_pressed(Window1))
        self.file_button.clicked.connect(lambda: self.selectFile(Window1))

        self.retranslateUi(Window1)
        QtCore.QMetaObject.connectSlotsByName(Window1)
        
        Window1.setStyleSheet("background-color: #437f8a ;\n" "color: white;")
        
        self.exit_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.submit_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.file_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )

    def retranslateUi(self, Window1):
        _translate = QtCore.QCoreApplication.translate
        Window1.setWindowTitle(_translate("Window1", "MainWindow"))
        self.title_label.setText(_translate("Window1", "COMPILER DESIGN PROJECT"))
        self.file_label.setText(_translate("Window1", "Insert File:"))
        self.file_button.setText(_translate("Window1", "Search for file"))
        self.submit_button.setText(_translate("Window1", "Submit"))
        self.exit_button.setText(_translate("Window1", "Exit"))


    def selectFile(self, Window1):
        #self.filepath
        fp = QFileDialog.getOpenFileName()
        self.filepath = fp[0].split("/")[-1]
        global postno
        postno = self.filepath
        print(self.filepath)

    def exit_button_pressed(self):
        sys.exit(app.exec_())

    def submit_button_pressed(self, window):
        tmp=subprocess.Popen("hashes.exe " + self.filepath,shell=True,stdout = subprocess.PIPE)
        tmp.wait()
        result = []
        for line in tmp.stdout:
            result.append(line)
        
        tmp=subprocess.Popen("mentions.exe " + self.filepath,shell=True,stdout = subprocess.PIPE)
        result = []
        for line in tmp.stdout:
            result.append(line)
        tmp.wait()
        
        tmp=subprocess.Popen("filter_comments.exe " + self.filepath,shell=True,stdout = subprocess.PIPE)
        result = []
        for line in tmp.stdout:
            result.append(line)
        tmp.wait()

        fd = open("post_no.txt","w")
        fd.write(self.filepath.split(".")[0])
        fd.close()
        
        import emotion_analysis
        
        print("Done")
        self.update_db()
        self.openWindow(2, window)
        
    
    def update_db(self):
        global conn
        post = self.filepath.split(".")[0]
        mfile = post+"_mentions.txt"
        hfile = post+"_hashtags.txt"
        mList = openfile(mfile)
        hList = openfile(hfile)
        updatedatabase(conn, mList, TABLES[0])
        print("MENTIONS updated")
        updatedatabase(conn, hList, TABLES[1])
        print("HAHSTAGS updated")

    def openWindow(self,wno, window):
        self.new_window = QtWidgets.QMainWindow()
        if wno == 2:
            self.ui = Ui_Window2()
        elif wno == 3:
            self.ui = Ui_Window3()
        elif wno == 4:
            self.ui = Ui_Window4()
        elif wno == 5:
            self.ui = Ui_Window5()
        self.ui.setupUi(self.new_window)
        self.new_window.show()

        window.close()

In [4]:
class Ui_Window2(object):

    def setupUi(self, Window2):
        Window2.setObjectName("Window2")
        Window2.resize(561, 482)

        self.centralwidget = QtWidgets.QWidget(Window2)
        self.centralwidget.setObjectName("centralwidget")
        self.feature_list = QtWidgets.QComboBox(self.centralwidget)
        self.feature_list.setGeometry(QtCore.QRect(260, 120, 231, 31))

        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.feature_list.setFont(font)
        self.feature_list.setObjectName("feature_list")
        self.feature_list.addItem("")
        self.feature_list.addItem("")
        self.feature_list.addItem("")
        self.choose_feature_label = QtWidgets.QLabel(self.centralwidget)
        self.choose_feature_label.setGeometry(QtCore.QRect(90, 100, 161, 71))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(16)
        font.setBold(True)
        font.setWeight(75)
        self.choose_feature_label.setFont(font) 
        self.choose_feature_label.setObjectName("choose_feature_label")
        self.confirm_button = QtWidgets.QPushButton(self.centralwidget)
        self.confirm_button.setGeometry(QtCore.QRect(230, 190, 91, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.confirm_button.setFont(font)
        self.confirm_button.setObjectName("confirm_button")
        self.exit_button = QtWidgets.QPushButton(self.centralwidget)
        self.exit_button.setGeometry(QtCore.QRect(440, 390, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.exit_button.setFont(font)
        self.exit_button.setObjectName("exit_button")
        Window2.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(Window2)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 561, 21))
        self.menubar.setObjectName("menubar")
        Window2.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(Window2)
        self.statusbar.setObjectName("statusbar")
        Window2.setStatusBar(self.statusbar)

        self.exit_button.clicked.connect(self.exit_button_pressed)
        self.confirm_button.clicked.connect(lambda: self.confirm_button_pressed(Window2))

        self.retranslateUi(Window2)
        QtCore.QMetaObject.connectSlotsByName(Window2)
        
        Window2.setStyleSheet("background-color: #437f8a ;\n" "color: white;")
        
        self.exit_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.confirm_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        

    def retranslateUi(self, Window2):
        _translate = QtCore.QCoreApplication.translate
        Window2.setWindowTitle(_translate("Window2", "MainWindow"))
        self.feature_list.setItemText(0, _translate("Window2", "Count Occurrence"))
        self.feature_list.setItemText(1, _translate("Window2", "Check Most Tagged"))
        self.feature_list.setItemText(2, _translate("Window2", "Check Post Emotion"))
        self.choose_feature_label.setText(_translate("Window2", "Choose Feature:"))
        self.confirm_button.setText(_translate("Window2", "Confirm"))
        self.exit_button.setText(_translate("Window2", "Exit"))

    def exit_button_pressed(self):
        sys.exit(app.exec_())

    def confirm_button_pressed(self, window):
        choice = self.feature_list.currentText()
        index = self.feature_list.findText(choice, QtCore.Qt.MatchFixedString)
        print(choice, index)
        self.openWindow(index+3, window)

    def openWindow(self,wno, window):
        self.new_window = QtWidgets.QMainWindow()
        if wno == 2:
            self.ui = Ui_Window2()
        elif wno == 3:
            self.ui = Ui_Window3()
        elif wno == 4:
            self.ui = Ui_Window4()
        elif wno == 5:
            self.ui = Ui_Window5()
        self.ui.setupUi(self.new_window)
        self.new_window.show()

        window.close()


In [5]:
class Ui_Window3(object):

    def setupUi(self, Window3):
        Window3.setObjectName("Window3")
        Window3.resize(561, 481)

        self.centralwidget = QtWidgets.QWidget(Window3)
        self.centralwidget.setObjectName("centralwidget")
        self.enter_label = QtWidgets.QLabel(self.centralwidget)
        self.enter_label.setGeometry(QtCore.QRect(110, 80, 131, 71))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(16)
        font.setBold(True)
        font.setWeight(75)
        self.enter_label.setFont(font)
        self.enter_label.setObjectName("enter_label")
        self.type_label = QtWidgets.QLabel(self.centralwidget)
        self.type_label.setGeometry(QtCore.QRect(160, 140, 71, 71))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(16)
        font.setBold(True)
        font.setWeight(75)
        self.type_label.setFont(font)
        self.type_label.setObjectName("type_label")
        self.type_list = QtWidgets.QComboBox(self.centralwidget)
        self.type_list.setGeometry(QtCore.QRect(250, 160, 171, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.type_list.setFont(font)
        self.type_list.setObjectName("type_list")
        self.type_list.addItem("")
        self.type_list.addItem("")
        self.find_button = QtWidgets.QPushButton(self.centralwidget)
        self.find_button.setGeometry(QtCore.QRect(220, 210, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.find_button.setFont(font)
        self.find_button.setObjectName("find_button")
        self.exit_button = QtWidgets.QPushButton(self.centralwidget)
        self.exit_button.setGeometry(QtCore.QRect(450, 380, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.exit_button.setFont(font)
        self.exit_button.setObjectName("exit_button")
        self.back_button = QtWidgets.QPushButton(self.centralwidget)
        self.back_button.setGeometry(QtCore.QRect(360, 380, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.back_button.setFont(font)
        self.back_button.setObjectName("back_button")
        self.key_input = QtWidgets.QLineEdit(self.centralwidget)
        self.key_input.setGeometry(QtCore.QRect(250, 100, 171, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.key_input.setFont(font)
        self.key_input.setObjectName("key_input")
        Window3.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(Window3)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 561, 21))
        self.menubar.setObjectName("menubar")
        Window3.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(Window3)
        self.statusbar.setObjectName("statusbar")
        Window3.setStatusBar(self.statusbar)

        self.exit_button.clicked.connect(self.exit_button_pressed)
        self.back_button.clicked.connect(lambda: self.back_button_pressed(Window3))
        self.find_button.clicked.connect(self.find_button_pressed)

        self.retranslateUi(Window3)
        QtCore.QMetaObject.connectSlotsByName(Window3)
        
        Window3.setStyleSheet("background-color: #437f8a ;\n" "color: white;")
        
        self.exit_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.back_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.find_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )

    def retranslateUi(self, Window3):
        _translate = QtCore.QCoreApplication.translate
        Window3.setWindowTitle(_translate("Window3", "MainWindow"))
        self.enter_label.setText(_translate("Window3", "Enter Key:"))
        self.type_label.setText(_translate("Window3", "Type:"))
        self.type_list.setItemText(0, _translate("Window3", "Mentions"))
        self.type_list.setItemText(1, _translate("Window3", "Hashtags"))
        self.find_button.setText(_translate("Window3", "Find"))
        self.exit_button.setText(_translate("Window3", "Exit"))
        self.back_button.setText(_translate("Window3", "Back"))

    def exit_button_pressed(self):
        sys.exit(app.exec_())

    def back_button_pressed(self, window):
        self.openWindow(2, window)

    def find_button_pressed(self):
        print("FIND PRESSED")
        global conn
        global TABLES
        choice = self.type_list.currentText()
        index = self.type_list.findText(choice, QtCore.Qt.MatchFixedString)
        key = self.key_input.text()

        count = countoccurrence(conn, key, TABLES[index])

        msg = QMessageBox()
        msg.setWindowTitle("Count Occurrence Result: ")
        msg.setText("Finished Searching Database:")

        # 4 types of icons: Warning, Critical, Question, Information
        msg.setIcon(QMessageBox.Information)
        #msg.setStandardButtons(QMessageBox.Cancel|QMessageBox.Retry|QMessageBox.Ok)
        msg.setDefaultButton(QMessageBox.Ok)
        msg.setInformativeText("Count of \'"+key+"\': "+str(count))

        #msg.setDetailedText("details and stuff")
        # msg.buttonClicked.connect(self.popup_button)

        msg.exec_()

        print("Count of \'%s\': %d"%(key,count))
    
    def openWindow(self, wno, window):
        self.new_window = QtWidgets.QMainWindow()
        if wno == 2:
            self.ui = Ui_Window2()
        elif wno == 3:
            self.ui = Ui_Window3()
        elif wno == 4:
            self.ui = Ui_Window4()
        elif wno == 5:
            self.ui = Ui_Window5()
        self.ui.setupUi(self.new_window)
        self.new_window.show()

        window.close()


In [6]:
class Ui_Window4(object):
    def setupUi(self, Window4):
        Window4.setObjectName("window4")
        Window4.resize(561, 484)

        self.centralwidget = QtWidgets.QWidget(Window4)
        self.centralwidget.setObjectName("centralwidget")
        self.type_label = QtWidgets.QLabel(self.centralwidget)
        self.type_label.setGeometry(QtCore.QRect(140, 40, 71, 71))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(16)
        font.setBold(True)
        font.setWeight(75)
        self.type_label.setFont(font)
        self.type_label.setObjectName("type_label")
        self.type_list = QtWidgets.QComboBox(self.centralwidget)
        self.type_list.setGeometry(QtCore.QRect(240, 60, 171, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.type_list.setFont(font)
        self.type_list.setObjectName("type_list")
        self.type_list.addItem("")
        self.type_list.addItem("")
        self.confirm_button = QtWidgets.QPushButton(self.centralwidget)
        self.confirm_button.setGeometry(QtCore.QRect(230, 110, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.confirm_button.setFont(font)
        self.confirm_button.setObjectName("confirm_button")
        self.back_button = QtWidgets.QPushButton(self.centralwidget)
        self.back_button.setGeometry(QtCore.QRect(350, 390, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.back_button.setFont(font)
        self.back_button.setObjectName("back_button")
        self.exit_button = QtWidgets.QPushButton(self.centralwidget)
        self.exit_button.setGeometry(QtCore.QRect(440, 390, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.exit_button.setFont(font)
        self.exit_button.setObjectName("exit_button")
        Window4.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(Window4)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 561, 21))
        self.menubar.setObjectName("menubar")
        Window4.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(Window4)
        self.statusbar.setObjectName("statusbar")
        Window4.setStatusBar(self.statusbar)

        self.exit_button.clicked.connect(self.exit_button_pressed)
        self.back_button.clicked.connect(lambda: self.back_button_pressed(Window4))
        self.confirm_button.clicked.connect(self.confirm_button_pressed)


        self.retranslateUi(Window4)
        QtCore.QMetaObject.connectSlotsByName(Window4)
        
        Window4.setStyleSheet("background-color: #437f8a ;\n" "color: white;")
        
        self.exit_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.back_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.confirm_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )

    def retranslateUi(self, window4):
        _translate = QtCore.QCoreApplication.translate
        window4.setWindowTitle(_translate("Window4", "MainWindow"))
        self.type_label.setText(_translate("Window4", "Type:"))
        self.type_list.setItemText(0, _translate("Window4", "Mentions"))
        self.type_list.setItemText(1, _translate("Window4", "Hashtags"))
        self.confirm_button.setText(_translate("Window4", "Confirm"))
        self.back_button.setText(_translate("Window4", "Back"))
        self.exit_button.setText(_translate("Window4", "Exit"))

    def exit_button_pressed(self):
        sys.exit(app.exec_())

    def back_button_pressed(self, window):
        self.openWindow(2, window)

    def confirm_button_pressed(self):
        print("CONFIRM PRESSED")
        global conn
        global TABLES
        choice = self.type_list.currentText()
        index = self.type_list.findText(choice, QtCore.Qt.MatchFixedString)

        mcontent, table = findmax(conn,TABLES[index])

        msg = QMessageBox()
        msg.setWindowTitle("Most Tagged Result: ")
        msg.setText("Finished Searching Database:")

        # 4 types of icons: Warning, Critical, Question, Information
        msg.setIcon(QMessageBox.Information)
        #msg.setStandardButtons(QMessageBox.Cancel|QMessageBox.Retry|QMessageBox.Ok)
        msg.setDefaultButton(QMessageBox.Ok)
        # Maximum occuring %s is \'%s\' with count of %d."%(TABLES[tchoice-1],mcontent[0],mcontent[1]))
        msg.setInformativeText("Maximum Occuring "+TABLES[index]+" is \'"+mcontent[0]+"\' with count of "+str(mcontent[1]) ) 

        msg.setDetailedText(table)
        # msg.buttonClicked.connect(self.popup_button)

        msg.exec_()



    def openWindow(self,wno, window):
        self.new_window = QtWidgets.QMainWindow()
        if wno == 2:
            self.ui = Ui_Window2()
        elif wno == 3:
            self.ui = Ui_Window3()
        elif wno == 4:
            self.ui = Ui_Window4()
        elif wno == 5:
            self.ui = Ui_Window5()
        self.ui.setupUi(self.new_window)
        self.new_window.show()

        window.close()


In [7]:
class Ui_Window5(object):
    def setupUi(self, Window5):
        Window5.setObjectName("MainWindow")
        Window5.resize(562, 481)

        self.centralwidget = QtWidgets.QWidget(Window5)
        self.centralwidget.setObjectName("centralwidget")
        self.emotion_label = QtWidgets.QLabel(self.centralwidget)
        self.emotion_label.setGeometry(QtCore.QRect(130, 60, 111, 71))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(16)
        font.setBold(True)
        font.setWeight(75)
        self.emotion_label.setFont(font)
        self.emotion_label.setObjectName("emotion_label")
        self.type_list = QtWidgets.QComboBox(self.centralwidget)
        self.type_list.setGeometry(QtCore.QRect(260, 80, 171, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.type_list.setFont(font)
        self.type_list.setObjectName("type_list")
        self.type_list.addItem("")
        self.type_list.addItem("")
        self.type_list.addItem("")
        self.confirm_button = QtWidgets.QPushButton(self.centralwidget)
        self.confirm_button.setGeometry(QtCore.QRect(230, 130, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.confirm_button.setFont(font)
        self.confirm_button.setObjectName("confirm_button")
        self.back_button = QtWidgets.QPushButton(self.centralwidget)
        self.back_button.setGeometry(QtCore.QRect(360, 380, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.back_button.setFont(font)
        self.back_button.setObjectName("back_button")
        self.exit_button = QtWidgets.QPushButton(self.centralwidget)
        self.exit_button.setGeometry(QtCore.QRect(450, 380, 75, 23))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(14)
        self.exit_button.setFont(font)
        self.exit_button.setObjectName("exit_button")
        self.predict_button = QtWidgets.QPushButton(self.centralwidget)
        self.predict_button.setGeometry(QtCore.QRect(200, 270, 111, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.predict_button.setFont(font)
        self.predict_button.setObjectName("predict_button")
        self.comment_label = QtWidgets.QLabel(self.centralwidget)
        self.comment_label.setGeometry(QtCore.QRect(70, 200, 161, 61))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(12)
        font.setBold(True)
        font.setWeight(75)
        self.comment_label.setFont(font)
        self.comment_label.setObjectName("comment_label")
        self.comment_input = QtWidgets.QLineEdit(self.centralwidget)
        self.comment_input.setGeometry(QtCore.QRect(240, 220, 171, 31))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(10)
        self.comment_input.setFont(font)
        self.comment_input.setObjectName("comment_input")
        
        
        Window5.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(Window5)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 562, 21))
        self.menubar.setObjectName("menubar")
        Window5.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(Window5)
        self.statusbar.setObjectName("statusbar")
        Window5.setStatusBar(self.statusbar)

        self.exit_button.clicked.connect(self.exit_button_pressed)
        self.back_button.clicked.connect(lambda: self.back_button_pressed(Window5))
        self.confirm_button.clicked.connect(self.confirm_button_pressed)
        self.predict_button.clicked.connect(self.predict_button_pressed)
        
        self.retranslateUi(Window5)
        QtCore.QMetaObject.connectSlotsByName(Window5)
        
        Window5.setStyleSheet("background-color: #437f8a ;\n" "color: white;")
        
        self.exit_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.back_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.predict_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )
        
        self.confirm_button.setStyleSheet(
        """
            QWidget {
                border: 0.5px solid white;
                border-radius: 5px;
            }

            QWidget:hover {
                border: 2px solid white;
            }
            
        """
        )

        
    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("Window5", "MainWindow"))
        self.emotion_label.setText(_translate("Window5", "Emotion:"))
        self.type_list.setItemText(0, _translate("Window5", "Balanced"))
        self.type_list.setItemText(1, _translate("Window5", "Optimistic"))
        self.type_list.setItemText(2, _translate("Window5", "Pesimistic"))
        self.confirm_button.setText(_translate("Window5", "Confirm"))
        self.back_button.setText(_translate("Window5", "Back"))
        self.exit_button.setText(_translate("Window5", "Exit"))
        self.predict_button.setText(_translate("Window5", "Predict Comment"))
        self.comment_label.setText(_translate("Window5", "Enter New Comment: "))

    def exit_button_pressed(self):
        sys.exit(app.exec_())

    def back_button_pressed(self, window):
        self.openWindow(2, window)

    def confirm_button_pressed(self):
        global postno
        
        choice = self.type_list.currentText()
        index = self.type_list.findText(choice, QtCore.Qt.MatchFixedString)
        
        print(postno, choice, index)
        self.display_comments(index)
        
    def predict_button_pressed(self):
        print("PREDICT PRESSED")
        comment = self.comment_input.text()
        print(comment)
        fd = open("new_comment.txt","w")
        fd.write(comment)
        fd.close()
        
        import likeprediction
        import importlib
        importlib.reload(likeprediction)
        
        file1 = open("new_comment_likes.txt", 'r')
        Lines = file1.readlines()
        file1.close()
        likes = Lines[0]
        print(likes)
        
        msg = QMessageBox()
        msg.setWindowTitle("Result: ")
        msg.setText("Finished Calculating Predicted number of likes: "+likes)

        # 4 types of icons: Warning, Critical, Question, Information
        #msg.setIcon(QMessageBox.Information)
        #msg.setStandardButtons(QMessageBox.Cancel|QMessageBox.Retry|QMessageBox.Ok)
        msg.setDefaultButton(QMessageBox.Ok)
        # Maximum occuring %s is \'%s\' with count of %d."%(TABLES[tchoice-1],mcontent[0],mcontent[1]))
        #msg.setInformativeText(table) 

        #msg.setDetailedText(table)
        # msg.buttonClicked.connect(self.popup_button)

        msg.exec_()
        
    def display_comments(self,flag):
        with open('output.csv', newline='\n') as csvfile:
            rec = []
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                temp = row[2]
                row[2] = row[0]
                row[0] = temp
                rec.append(row)

        #flag = 0
        new_rec = []
        for i in range(1,len(rec)):
            if flag == 1:
                if int(rec[i][3]) > 0:
                    new_rec.append(rec[i][:-1])
            elif flag == 0:
                if int(rec[i][3]) == 0:
                    new_rec.append(rec[i][:-1])
            else:
                if int(rec[i][3]) < 0:
                    new_rec.append(rec[i][:-1])

        #table = tabulate(new_rec, headers=rec[0][:-1], numalign="right")
        table = tabulate(new_rec)
        #d_table = str(table)
        #print(table)
        
        msg = QMessageBox()
        msg.setStyleSheet("QLabel{min-width: 600px;}");
        msg.setWindowTitle("Result: ")
        msg.setText("Finished Searching Database:")

        # 4 types of icons: Warning, Critical, Question, Information
        #msg.setIcon(QMessageBox.Information)
        #msg.setStandardButtons(QMessageBox.Cancel|QMessageBox.Retry|QMessageBox.Ok)
        msg.setDefaultButton(QMessageBox.Ok)
        # Maximum occuring %s is \'%s\' with count of %d."%(TABLES[tchoice-1],mcontent[0],mcontent[1]))
        msg.setInformativeText(table) 

        #msg.setDetailedText(table)
        # msg.buttonClicked.connect(self.popup_button)

        msg.exec_()
        
    def openWindow(self,wno, window):
        self.new_window = QtWidgets.QMainWindow()
        if wno == 2:
            self.ui = Ui_Window2()
        elif wno == 3:
            self.ui = Ui_Window3()
        elif wno == 4:
            self.ui = Ui_Window4()
        elif wno == 5:
            self.ui = Ui_Window5()
        self.ui.setupUi(self.new_window)
        self.new_window.show()
        window.close()




In [ ]:
if __name__ == "__main__":
    global TABLES
    TABLES = ["mentions", "hashtags"]
    file = "compiler_project.db"
    
    global conn
    conn = opendatabase(file)

    app = QtWidgets.QApplication(sys.argv)
    Window1 = QtWidgets.QMainWindow()
    ui = Ui_Window1()
    ui.setupUi(Window1)
    Window1.show()
    sys.exit(app.exec_())

5.txt
5

line:  16; The  best day in Karans show   kj;1

Comment:The  best day in Karans show   kj
Score:3.0

line:  14; had better plans with  har n12  founders23 APPLE;2

Comment:had better plans with  har n12  founders23 APPLE
Score:2.0

line:  10;  design  rules the world isn't it  sammer   undou bted come on agree Sameer ;3

Comment:design  rules the world isn't it  sammer   undou bted come on agree Sameer
Score:1.0

line:  18; fun fact it is  necessary to have a  space  endl after each comment ;4

Comment:fun fact it is  necessary to have a  space  endl after each comment
Score:4.0

line:  4; Nlp well that was on  purpose start of sentence ;5

Comment:Nlp well that was on  purpose start of sentence
Score:-2.0

line:  0; Its indeed a harsh  hit   sad;6

Comment:Its indeed a harsh  hit   sad
Score:-4.0

line:  12; That was an  extremely  powerful shot  reynold;7

Comment:That was an  extremely  powerful shot  reynold
Score:2.0

line:  0; has always been keen on exhibiting his skill